# Custom CNN architecture

In [1]:
# Run the previous notebook to load all its classes and functions
%run busi_augmentation_2.ipynb

Found 210 images and 211 masks in malignant folder


Processing malignant masks: 100%|███████████████████████████████████████████████████| 211/211 [00:00<00:00, 571.69it/s]


Found 133 images and 133 masks in normal folder


Processing normal masks: 100%|██████████████████████████████████████████████████████| 133/133 [00:00<00:00, 262.71it/s]


Found 437 images and 454 masks in benign folder


Processing benign masks: 100%|██████████████████████████████████████████████████████| 454/454 [00:00<00:00, 604.84it/s]


Combined 2 masks for malignant (53).png
Combined 2 masks for benign (100).png
Combined 2 masks for benign (163).png
Combined 2 masks for benign (173).png
Combined 2 masks for benign (181).png
Combined 3 masks for benign (195).png
Combined 2 masks for benign (25).png
Combined 2 masks for benign (315).png
Combined 2 masks for benign (346).png
Combined 2 masks for benign (4).png
Combined 2 masks for benign (424).png
Combined 2 masks for benign (54).png
Combined 2 masks for benign (58).png
Combined 2 masks for benign (83).png
Combined 2 masks for benign (92).png
Combined 2 masks for benign (93).png
Combined 2 masks for benign (98).png
Dataset shape: (780, 224, 224, 1)
Masks shape: (780, 224, 224, 1)
Labels shape: (780,)
Class distribution: Normal: 133, Benign: 437, Malignant: 210
Training set sizes: X_train: (546, 224, 224, 1), y_train: (546,), masks: (780, 224, 224, 1)
Augmented training set sizes: X_aug: (1758, 224, 224, 1), y_aug: (1758,), masks: (1758, 224, 224, 1)
Validation set sizes

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, LeakyReLU, BatchNormalization, Conv2D, MaxPooling2D, Flatten,  Dropout
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.datasets import mnist, cifar10
from tensorflow.keras.callbacks import EarlyStopping

import seaborn as sns
import time

In [3]:
print(X_aug.shape)
print(y_aug.shape)
print(masks_aug.shape)

(1758, 224, 224, 1)
(1758,)
(1758, 224, 224, 1)


# Apply MINMAX normalization (suitable for CNNs custom)

In [4]:
# Train
X_train_aug = X_aug.astype("float32") / 255.0
masks_train_aug = masks_aug.astype("float32") / 255.0
y_train_aug = y_aug

# Test
X_test = X_test.astype("float32") / 255.0
masks_test = masks_test.astype("float32") / 255.0
y_test = y_test

# Validation
X_val = X_val.astype("float32") / 255.0
masks_val = masks_val.astype("float32") / 255.0
y_val = y_val



# 1. Model from article

In [8]:
model = Sequential([
    # Input layer (grayscale images 224x224x1)
    Input(shape=(224, 224, 1)),

    # Convolutional layer: 20 filters, kernel size 5x5
    Conv2D(20, (5, 5), padding="same"),
    
    # Batch Normalization (20 channels)
    BatchNormalization(),
    
    # ReLU activation
    Activation("relu"),
    
    # MaxPooling
    MaxPooling2D(pool_size=(2, 2)),
    
    # Flatten before fully connected
    Flatten(),
    
    # Fully connected layer (let’s use 128 units)
    Dense(128, activation="relu"),
    
    # Dropout 50%
    Dropout(0.5),
    
    # Output layer with 3 classes
    Dense(3, activation="softmax")
])

In [9]:
# Compile
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train
history = model.fit(
    X_train_aug, y_train_aug,
    validation_data=(X_val, y_val),
    epochs=15,
    batch_size=32,
    callbacks=[early_stop]
)

# Save
label = (
    f"custom_CNN_from_Convolutionalneuralnetwork-basedmodelsfordiagnosisofbreast"
)
histories.append((label, history))

Epoch 1/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 38s 662ms/step - accuracy: 0.3490 - loss: 45.4109 - val_accuracy: 0.2735 - val_loss: 1.0989
Epoch 2/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 36s 645ms/step - accuracy: 0.3522 - loss: 1.1493 - val_accuracy: 0.5641 - val_loss: 1.0962
Epoch 3/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 36s 651ms/step - accuracy: 0.3508 - loss: 1.0980 - val_accuracy: 0.5556 - val_loss: 1.0943
Epoch 4/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 36s 648ms/step - accuracy: 0.3435 - loss: 1.0986 - val_accuracy: 0.5556 - val_loss: 1.0932
Epoch 5/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 36s 647ms/step - accuracy: 0.3555 - loss: 1.0973 - val_accuracy: 0.5556 - val_loss: 1.0898
Epoch 6/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 35s 644ms/step - accuracy: 0.3487 - loss: 1.0981 - val_accuracy: 0.5556 - val_loss: 1.0892
Epoch 7/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 36s 646ms/step - accuracy: 0.3493 - loss: 1.0978 - val_accuracy: 0.5556 - val_loss: 1.0876
Epoch 8/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 35s 643ms/step - accuracy: 0.3609 - loss: 1.0974 - val_acc

### Apply early stopping to save time & avoid overfitting

In [5]:
# Early stopping
early_stop = EarlyStopping(patience=5, restore_best_weights=True,monitor='val_loss', verbose=1)

# Custom CNNs (initial trials)

In [5]:
# Hyperparameter options

activations = [
    ("relu", "relu"),
    ("leakyrelu", LeakyReLU(negative_slope=0.01))
]
dense_layers =[32, 64, 128]


histories = []


for dense_nr in dense_layers:
    for activation_name, activation_fn in activations:
            print(
                f"Training model with: "
                f"activation={activation_name}, "
                f"dense_layer={dense_nr}"
            )


            # Model definition
            model = Sequential([
                Input(shape=(224, 224, 1)),
                Conv2D(dense_nr, kernel_size=(3,3), activation=activation_fn),
                MaxPooling2D(pool_size=(2, 2)),

                Conv2D(dense_nr*2, kernel_size=(3,3), activation=activation_fn),
                MaxPooling2D(pool_size=(2, 2)),

                Conv2D(dense_nr*4, kernel_size=(3,3), activation=activation_fn),
                MaxPooling2D(pool_size=(2, 2)),

                Flatten(),
                Dense(128, activation=activation_fn),
                Dropout(0.5),
                Dense(3, activation='softmax')
            ])

            # Compile
            model.compile(
                optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']
            )

            # Train
            history = model.fit(
                X_train_aug, y_train_aug,
                validation_data=(X_val, y_val),
                epochs=15,
                batch_size=32,
                callbacks=[early_stop]
            )

            # Save
            label = (
                f"dense_nr={dense_nr}, "
                f"activation={activation_name} "
            )
            histories.append((label, history))


Training model with: activation=relu, dense_layer=32
Epoch 1/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 32s 535ms/step - accuracy: 0.3700 - loss: 1.2998 - val_accuracy: 0.5812 - val_loss: 0.9757
Epoch 2/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 29s 523ms/step - accuracy: 0.5673 - loss: 0.9334 - val_accuracy: 0.5726 - val_loss: 0.8272
Epoch 3/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 29s 522ms/step - accuracy: 0.7537 - loss: 0.6324 - val_accuracy: 0.6496 - val_loss: 0.7889
Epoch 4/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 29s 524ms/step - accuracy: 0.8248 - loss: 0.4505 - val_accuracy: 0.5983 - val_loss: 0.9415
Epoch 5/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 29s 525ms/step - accuracy: 0.9171 - loss: 0.2377 - val_accuracy: 0.6410 - val_loss: 1.1813
Epoch 6/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 29s 525ms/step - accuracy: 0.9516 - loss: 0.1418 - val_accuracy: 0.6667 - val_loss: 1.2107
Epoch 7/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 29s 527ms/step - accuracy: 0.9546 - loss: 0.1256 - val_accuracy: 0.6325 - val_loss: 1.5552
Epoch 8/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 29s 53

Best performer: 
- Input → Conv2D(64) → MaxPool →
- Conv2D(128) → MaxPool →
- Conv2D(256) → MaxPool →
- Dense(128) + Dropout → Dense(3, softmax)
- Activation: LeakyReLU

and balances speed, performance, and complexity.


Let us try fine-tuning the model, with changing some parameters and adding small changes that could improve performance.

# 1. Small changes (different dropouts and initial kernels)

### Let us keep a history also for this model adjustments to see the plots

In [6]:
histories_leaky_128 = []

And now train the models

In [ ]:
initial_kernels = [(3,3),(5,5)]
dropouts = [0.5, 0.6, 0.7]


for initial_kernel in initial_kernels:
    for dropout in dropouts:
        print(
                f"Training model with: "
                f"initial_kernel={initial_kernel}, "
                f"dropout={dropout}"
            )


        # Model definition
        model = Sequential([
            Input(shape=(224, 224, 1)),
            Conv2D(64, kernel_size=initial_kernel, activation=(LeakyReLU(negative_slope=0.01))),
            MaxPooling2D(pool_size=(2, 2)),
        
            Conv2D(128, kernel_size=(3,3), activation=(LeakyReLU(negative_slope=0.01))),
            MaxPooling2D(pool_size=(2, 2)),
        
            Conv2D(256, kernel_size=(3,3), activation=(LeakyReLU(negative_slope=0.01))),
            MaxPooling2D(pool_size=(2, 2)),
        
            Flatten(),
            Dense(128, activation=(LeakyReLU(negative_slope=0.01))),
            Dropout(dropout),
            Dense(3, activation='softmax')
        ])
        
        # Compile
        model.compile(
            optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        
        # Train
        history = model.fit(
            X_train_aug, y_train_aug,
            validation_data=(X_val, y_val),
            epochs=15,
            batch_size=32,
            callbacks=[early_stop]
        )
        
        # Save
        label = (
            f"initial_kernel={initial_kernel}, "
            f"dropout={dropout}"
        )
        histories_leaky_128.append((label, history))

Training model with: initial_kernel=(3, 3), dropout=0.5
Epoch 1/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - accuracy: 0.3580 - loss: 1.4246 - val_accuracy: 0.6068 - val_loss: 1.0028
Epoch 2/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 94s 2s/step - accuracy: 0.4897 - loss: 1.0140 - val_accuracy: 0.6154 - val_loss: 0.8748
Epoch 3/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - accuracy: 0.6042 - loss: 0.8529 - val_accuracy: 0.6239 - val_loss: 0.8414
Epoch 4/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - accuracy: 0.7346 - loss: 0.6558 - val_accuracy: 0.6923 - val_loss: 0.7376
Epoch 5/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 93s 2s/step - accuracy: 0.8433 - loss: 0.4022 - val_accuracy: 0.6838 - val_loss: 0.8199
Epoch 6/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 94s 2s/step - accuracy: 0.9154 - loss: 0.2183 - val_accuracy: 0.6838 - val_loss: 0.9220
Epoch 7/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - accuracy: 0.9530 - loss: 0.1212 - val_accuracy: 0.7094 - val_loss: 1.0271
Epoch 8/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - accuracy: 

In [ ]:
initial_kernels = [(5,5)]
dropouts = [0.5, 0.6, 0.7]


for initial_kernel in initial_kernels:
    for dropout in dropouts:
        print(
                f"Training model with: "
                f"initial_kernel={initial_kernel}, "
                f"dropout={dropout}"
            )


        # Model definition
        model = Sequential([
            Input(shape=(224, 224, 1)),
            Conv2D(64, kernel_size=initial_kernel, activation=(LeakyReLU(negative_slope=0.01))),
            MaxPooling2D(pool_size=(2, 2)),
        
            Conv2D(128, kernel_size=(3,3), activation=(LeakyReLU(negative_slope=0.01))),
            MaxPooling2D(pool_size=(2, 2)),
        
            Conv2D(256, kernel_size=(3,3), activation=(LeakyReLU(negative_slope=0.01))),
            MaxPooling2D(pool_size=(2, 2)),
        
            Flatten(),
            Dense(128, activation=(LeakyReLU(negative_slope=0.01))),
            Dropout(dropout),
            Dense(3, activation='softmax')
        ])
        
        # Compile
        model.compile(
            optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        
        # Train
        history = model.fit(
            X_train_aug, y_train_aug,
            validation_data=(X_val, y_val),
            epochs=15,
            batch_size=32,
            callbacks=[early_stop]
        )
        
        # Save
        label = (
            f"initial_kernel={initial_kernel}, "
            f"dropout={dropout}"
        )
        histories_leaky_128.append((label, history))

In [ ]:
for label, history in histories:
    plt.figure(figsize=(40, 8))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Loss '+label)
    plt.plot(history.history['val_loss'], label='Val Loss '+label)
    plt.legend()
    plt.title("Loss over Epochs")
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.legend()
    plt.title("Accuracy over Epochs")
    plt.show()

